In [15]:
from selenium import webdriver
import requests
from parsel import Selector

## 1) We obtain 10000 url of the properties with web driver.

In [12]:
driver = webdriver.Chrome(executable_path='chromedriver.exe')

# We add url of each house in 333 page in immoweb to houses_url list.
houses_url = []

for i in range(1, 334):
    # We used 'i' to build urls of the 333 page in immoweb.
    #   So, we can reach 333 pages with for loop.
    apikey = str(i)+'&orderBy=relevance'
    url = 'https://www.immoweb.be/en/search/house/for-sale?countries=BE&page='+apikey

    # An implicit wait tells WebDriver to poll the DOM for a
    #   certain amount of time when trying to find any element 
    #     (or elements) not immediately available. 
    driver.implicitly_wait(10)
    
    # The first thing you’ll want to do with WebDriver is navigate
    #   to a link. The normal way to do this is by calling get method:    
    driver.get(url)

    # Selector` allows you to select parts of an XML or HTML text using CSS
    #   or XPath expressions and extract data from it.
    sel = Selector(text=driver.page_source) 

    # xpath query of the houses in the immoweb page
    xpath_houeses = '//*[@id="main-content"]/li//h2//a/@href'
    
    # Find nodes matching the xpath ``query`` and return the result
    page_houses_url = sel.xpath(xpath_houeses).extract()
    
    # There are approximately 30 houses in each page.
    # We add each page url list to houses_url like matrix.
    houses_url.append(page_houses_url)

print(houses_url[2][4])
print(len(houses_url))

https://www.immoweb.be/en/classified/town-house/for-sale/strombeek-bever/1853/8911430?searchId=5f6c49339213c
333


## 2) We scrape all data of each house with requests

In [ ]:
for page_list in houses_url:
    for url_a_house in page_list:
        r = requests.get(url_a_house)
        html_doc = r.text
        .
        .
        .
        .
        .
        .

## 3) We store all data to a csv file with dataframe.

In [ ]:
import pandas as pd
houses_dict = {'Locality':locality, 'Type of property':type_property ,
               'Subtype of property':subtype, 'Price':price,
               'Type of sale':type_sale, 'Number of rooms': num_rooms ,
               'Area':area, 'Fully equipped kitchen':kitchen ,
               'Furnished':furnished, 'Open fire':fire,
               'Terrace':terrace_area, 'Garden':garden_area,
               'Surface of the land':land, 'Surface area of the plot of land':land_plot,
               'Number of facades':num_facade, 'Swimming pool':pool,
               'State of the building':state}
    
df = pd.DataFrame(houses_dict)
df.to_csv('all_data_of_the_houses.csv')